# MirrorX Zero-Cost Virtual Try-On

**Run virtual try-on for FREE using Google Colab's GPU!**

This notebook sets up:
- OOTDiffusion for virtual try-on
- InsightFace for face preservation
- Gradio API for remote access

**Cost: $0** (uses Colab's free T4 GPU)

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q torch torchvision
!pip install -q transformers diffusers accelerate
!pip install -q insightface onnxruntime-gpu
!pip install -q gradio Pillow opencv-python numpy

In [ ]:
# Install OOTDiffusion
!pip install -q git+https://github.com/levihsu/OOTDiffusion.git

In [ ]:
# Download face swapper model
!mkdir -p models
!wget -q -O models/inswapper_128.onnx https://huggingface.co/deepinsight/inswapper/resolve/main/inswapper_128.onnx

In [ ]:
import torch
import gradio as gr
from PIL import Image
import numpy as np
import io
import base64
import time

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Load models
print("Loading OOTDiffusion...")
from ootdiffusion import OOTDiffusionPipeline

ootd_model = OOTDiffusionPipeline.from_pretrained(
    "levihsu/OOTDiffusion",
    torch_dtype=torch.float16
).to("cuda")

print("OOTDiffusion loaded!")

In [ ]:
# Load InsightFace for face preservation
print("Loading InsightFace...")
from insightface.app import FaceAnalysis
import insightface

face_app = FaceAnalysis(
    name="buffalo_l",
    providers=['CUDAExecutionProvider']
)
face_app.prepare(ctx_id=0, det_size=(640, 640))

# Load face swapper
face_swapper = insightface.model_zoo.get_model(
    'models/inswapper_128.onnx',
    providers=['CUDAExecutionProvider']
)

print("InsightFace loaded!")

In [ ]:
def preserve_face(original_np, generated_np):
    """Swap original face into generated image."""
    try:
        original_faces = face_app.get(original_np)
        generated_faces = face_app.get(generated_np)
        
        if not original_faces or not generated_faces:
            return generated_np
        
        result = face_swapper.get(
            generated_np,
            generated_faces[0],
            original_faces[0],
            paste_back=True
        )
        return result
    except Exception as e:
        print(f"Face preservation failed: {e}")
        return generated_np


def virtual_tryon(
    person_image,
    clothing_image,
    category="upperbody",
    preserve_face_flag=True,
    num_steps=20,
    guidance_scale=2.0
):
    """Generate virtual try-on with face preservation."""
    start = time.time()
    
    # Resize images
    person_image = person_image.resize((768, 1024), Image.Resampling.LANCZOS)
    clothing_image = clothing_image.resize((768, 1024), Image.Resampling.LANCZOS)
    
    # Generate try-on
    result = ootd_model(
        model_image=person_image,
        cloth_image=clothing_image,
        category=category,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
    )
    result_image = result.images[0]
    
    # Preserve face
    if preserve_face_flag:
        person_np = np.array(person_image)
        result_np = np.array(result_image)
        result_np = preserve_face(person_np, result_np)
        result_image = Image.fromarray(result_np)
    
    elapsed = time.time() - start
    return result_image, f"Generated in {elapsed:.2f}s"

print("Functions defined!")

In [ ]:
# Create Gradio interface with public URL
demo = gr.Interface(
    fn=virtual_tryon,
    inputs=[
        gr.Image(type="pil", label="Your Photo"),
        gr.Image(type="pil", label="Clothing"),
        gr.Dropdown(["upperbody", "lowerbody", "dress"], value="upperbody", label="Category"),
        gr.Checkbox(value=True, label="Preserve Face"),
        gr.Slider(10, 30, value=20, step=5, label="Quality Steps"),
        gr.Slider(1.0, 5.0, value=2.0, step=0.5, label="Guidance Scale"),
    ],
    outputs=[
        gr.Image(type="pil", label="Result"),
        gr.Textbox(label="Status"),
    ],
    title="MirrorX Virtual Try-On (FREE)",
    description="Upload your photo and clothing to see how it looks on you!",
)

# Launch with public URL (share=True creates a public link)
demo.launch(share=True, debug=True)

## Using the API

Once launched, copy the public URL (e.g., `https://xxxxx.gradio.live`) and set it as:

```
SELF_HOSTED_TRYON_ENDPOINT=https://xxxxx.gradio.live/api/predict
```

Then call from your MirrorX backend:

```typescript
const response = await fetch('https://xxxxx.gradio.live/api/predict', {
  method: 'POST',
  headers: { 'Content-Type': 'application/json' },
  body: JSON.stringify({
    data: [personImageBase64, clothingImageBase64, 'upperbody', true, 20, 2.0]
  })
});
```